source:
https://keras.io/examples/nlp/neural_machine_translation_with_transformer/#setup

In [1]:
import random
import keras_nlp
import numpy as np
import tensorflow as tf


In [2]:
SEQ_LENGTH = 1024
BATCH_SIZE = 32

Dataset preprocessing

In [3]:
data = 'alignmentsTRAIN'

with open(data) as f:
    lines = f.readlines()
pairs = []
for line in lines:
    unaligned, aligned, score = line.split('\t')
    aligned = 'START ' + aligned + ' END'
    pairs.append((unaligned, aligned))

In [4]:
random.shuffle(pairs)

train_pairs = pairs[:1000]
val_pairs = pairs[1000:1200]
test_pairs = pairs[1200:]

In [5]:
input_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = './data/input_dict',
    sequence_length = SEQ_LENGTH,
    oov_token = '[UNK]'
)

output_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary = './data/output_dict',
    sequence_length = SEQ_LENGTH + 1,
    oov_token = '[UNK]'
)

In [6]:
def format_dataset(unaligned, aligned):
    unaligned = input_tokenizer(unaligned)
    aligned = output_tokenizer(aligned)
    return ({"encoder_inputs": unaligned, "decoder_inputs": aligned[:, :-1],}, aligned[:, 1:])


def make_dataset(pairs):
    unaligned, aligned = zip(*pairs)
    unaligned = list(unaligned)
    aligned = list(aligned)
    dataset = tf.data.Dataset.from_tensor_slices((unaligned, aligned))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (32, 1024)
inputs["decoder_inputs"].shape: (32, 1024)
targets.shape: (32, 1024)


Model building

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [9]:
embed_dim = 256
latent_dim = 2048
num_heads = 16

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(SEQ_LENGTH, 24, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(SEQ_LENGTH, 445, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(445, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.save('model')

In [ ]:
transformer_loaded = keras.models.load_model('model')

Training phase

In [ ]:
epochs = 1

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

In [20]:
transformer.save_weights('weight1')

In [ ]:
transformer_loaded = keras.models.load_model('model')

In [ ]:
max_length = 128


def decode_sequence(input_pair):
    tokenized_input_pair = input_tokenizer([input_pair])
    alignment = "START"
    for i in range(max_length):
        tokenized_target = output_tokenizer([alignment])[:, :-1]
        predictions = transformer([tokenized_input_pair, tokenized_target])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = output_tokenizer.id_to_token(sampled_token_index)
        alignment += " " + sampled_token

        if sampled_token == "END":
            break
    return alignment

unaligned = [pair[0] for pair in test_pairs]
for _ in range(30):
    input = random.choice(unaligned)
    print(input)
    aligned = decode_sequence(input)
    print(aligned)

In [25]:
output_tokenizer.id_to_token(5)

'Ax'